In [1]:
import sys
import os
from datetime import datetime
import json
import shutil
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm_notebook, tqdm
from matplotlib import pyplot as plt

sys.path.append('../src/')
from env import BTCMarket_Env
from agent import Trader_Agent
from DQNtrainer import DQNTrainer
from DRLtrainer import DRLTrainer
from collections import deque


2023-03-12 16:04:26.049338: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import agent as TAgent
import env as BTCEnv

from importlib import reload 

In [3]:
obs_space = (8,20)
act_space = 1

money = 20000
fee = 0.001
episodes = 1
runs_p_eps = 1
TAgent=reload(TAgent)
BTCEnv=reload(BTCEnv)
env = BTCEnv.BTCMarket_Env(observation_space = obs_space,
            action_space = act_space,
            start_money = money,
            trading_fee= fee)
agent = TAgent.Trader_Agent(observation_space = obs_space,
            action_space = act_space,
            epsilon = 0.01)
drltrainer = DRLTrainer(env, agent,
                observation_space = obs_space,
                action_space = act_space,
                batch_size=32,                        
                lstm_path="./best_models/11_mar_2023/lstm_2.h5")

# drltrainer.rollout(n_episodes=episodes, run_per_episode=runs_p_eps)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 160)               0         
                                                                 
 dense (Dense)               (None, 256)               41216     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 82,433
Trainable params: 82,433
Non-trainable params: 0
_________________________________________________________________
Model Loss: <DRLtrainer.DRLLossFunctions object at 0

2023-03-12 16:04:35.980599: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 16:04:35.981448: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
env.reset()
data_samples = env.episode_length
memory = deque()
for t in range(20):
    state, _, _ = env.step(np.array([0]))

In [5]:
state.shape

(1, 20, 8)

In [12]:
agent.model.predict(state, verbose=1)[0]

1/1 [==============================] - 0s 31ms/step


array([0.52278113], dtype=float32)

In [ ]:
env.reset()
data_samples = env.episode_length
memory = deque()

state, _, _ = env.step(np.array([0]))
for t in tqdm(range(data_samples)):
    # Compute Action
    tmp_wallet_value = env.wallet_value
    action = agent.compute_action(state)
    # Compute new step
    next_state, reward, done = env.step(action=action)
    # save Experience to Memory
    memory.append((state, action, reward, next_state, done))
    state = next_state

In [18]:
env.wallet_value

array([22234.95])

In [6]:
batch_size = 50


batch = []
# for i in range(1):
for i in range(max([len(memory) - batch_size,0]), len(memory)):
    batch.append(memory[i])

# Change here to fit new RL-Algo
# for index in range(0,len(batch)-1):

x_train = np.zeros((batch_size, 20,8))
y_train = np.zeros((batch_size, 4))
print(len(batch))


state, _, reward, next_state , done = batch[0]
action = agent.model.predict(state,verbose = 0)
for index in range(1,len(batch)):
    # Compute Reward Decay for DQN
    action_next = agent.model.predict(next_state,verbose = 0)
#     if not done:
#         reward += dqntrainer.gamma * np.max(action_next)
        
    if np.any(np.isnan(reward)) or np.any(np.isnan(action)):
        print(reward)
        print(action)
    target = np.nan_to_num(action,nan=0.0)      
    id_act = np.argmax(target)
    target[0,id_act] = reward
    
    target = np.nan_to_num(target,nan=0.0)
    state = np.nan_to_num(state,nan=0.0)
    x_train[index]= state[0]
    y_train[index]= target[0]
    
    
    # update state, action, reward for training
    state, _, reward, next_state , done = batch[index]
    action = action_next


result=drltrainer.agent.model.fit(x_train, y_train, epochs=5, verbose=1)
# agent.update_epsilon()


50
[nan]
[[0.53175]]
[nan]
[[0.48605838]]
[nan]
[[0.5748076]]
[nan]
[[0.5367304]]
[nan]
[[0.46627843]]
[nan]
[[0.45344213]]
[nan]
[[0.49437934]]
[nan]
[[0.42137742]]
[nan]
[[0.44183114]]
[nan]
[[0.49554604]]
[nan]
[[0.42949927]]
[nan]
[[0.41737896]]
[nan]
[[0.47609264]]
[nan]
[[0.44837612]]
[nan]
[[0.4503274]]
[nan]
[[0.5113878]]
[nan]
[[0.4983522]]
[nan]
[[0.47565052]]
[nan]
[[0.4914827]]
[nan]
[[0.41708475]]
[nan]
[[0.40020627]]
[nan]
[[0.43556508]]
[nan]
[[0.4912107]]
[nan]
[[0.52390206]]
[nan]
[[0.5288155]]
[nan]
[[0.51309913]]
[nan]
[[0.45242944]]
[nan]
[[0.4105394]]
[nan]
[[0.40095982]]
[nan]
[[0.32136416]]
[nan]
[[0.37901244]]
[nan]
[[0.42903683]]
[nan]
[[0.41149423]]
[nan]
[[0.37230483]]
[nan]
[[0.34529626]]
[nan]
[[0.4195807]]
[nan]
[[0.40654886]]
[nan]
[[0.4716751]]
[nan]
[[0.5065589]]
[nan]
[[0.431485]]
[nan]
[[0.5309753]]
[nan]
[[0.55155677]]
[nan]
[[0.43159816]]
[nan]
[[0.49637097]]
[nan]
[[0.4248774]]
[nan]
[[0.52509236]]
[nan]
[[0.4247751]]
[nan]
[[0.44242832]]
[nan]
[[0